In [1]:
import pandas as pd
import os
import json

In [2]:
def preprocess_wellness_csv(participant_id):
    wellness = pd.read_csv(f"./PMDATA/{participant_id}/pmsys/wellness.csv")
    wellness["effective_time_frame"] = pd.to_datetime(wellness["effective_time_frame"])
    wellness["effective_time_frame"] = wellness["effective_time_frame"].dt.date
    wellness.rename(columns = {"effective_time_frame":"dateTime"}, inplace = True)
    
    wellness[wellness["dateTime"].duplicated()]
    wellness["dateTime"].drop_duplicates(inplace = True)
    wellness.drop("soreness_area", axis = 1, inplace = True)

    return wellness

In [3]:
def preprocess_srpe_data(participant_id):
    srpe_df = pd.read_csv(f"./PMDATA/{participant_id}/pmsys/srpe.csv")

    # 1. Convert 'end_date_time' to datetime and extract date
    srpe_df['end_date_time'] = pd.to_datetime(srpe_df['end_date_time'].str.replace('Z', ''))
    srpe_df['srpe_date'] = srpe_df['end_date_time'].dt.date

    # 2. Calculate Session Training Load (sRPE)
    srpe_df['perceived_exertion'] = pd.to_numeric(srpe_df['perceived_exertion'], errors='coerce')
    srpe_df['duration_min'] = pd.to_numeric(srpe_df['duration_min'], errors='coerce')
    srpe_df['session_load_srpe'] = srpe_df['perceived_exertion'] * srpe_df['duration_min']

    # 3. Handle 'activity_names' (it's a string representation of a list, so convert it)
    import ast
    srpe_df['activity_names_list'] = srpe_df['activity_names'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

    # 4. Aggregate to Daily Level
    daily_srpe_metrics = srpe_df.groupby('srpe_date').agg(
        total_daily_training_load_srpe=('session_load_srpe', 'sum'),
        total_daily_duration_min=('duration_min', 'sum'),
        avg_daily_perceived_exertion=('perceived_exertion', 'mean'),
        num_daily_srpe_sessions=('end_date_time', 'count'), 
        unique_daily_activity_names=('activity_names_list', lambda x: list(set([item for sublist in x for item in sublist])))
    ).reset_index()

    # Rename date column for merging consistency
    daily_srpe_metrics.rename(columns={'srpe_date': 'dateTime'}, inplace=True)

    # Fill NaNs for sum-based metrics with 0 where appropriate (if no training on a day)
    daily_srpe_metrics.fillna({
        'total_daily_training_load_srpe': 0,
        'total_daily_duration_min': 0,
        'avg_daily_perceived_exertion': 0 # Or NaN if you prefer to distinguish no training vs 0 exertion
    }, inplace=True)

    return daily_srpe_metrics

In [4]:
def preprocess_reporting_data(participant_id):
    reporting_df = pd.read_csv(f"./PMDATA/{participant_id}/googledocs/reporting.csv")

    # 1. Convert 'date' and 'timestamp' columns
    reporting_df['date'] = pd.to_datetime(reporting_df['date'], format='%d/%m/%Y')
    reporting_df['dateTime'] = reporting_df['date'].dt.date

    # 2. Process 'meals' column to create boolean flags for each meal type
    meal_types = ['Breakfast', 'Lunch', 'Dinner', 'Evening']
    for meal in meal_types:
        reporting_df[f'had_{meal.lower()}'] = reporting_df['meals'].str.contains(meal, case=False, na=False)

    # 3. Process 'alcohol_consumed' column
    reporting_df['alcohol_consumed_flag'] = reporting_df['alcohol_consumed'].apply(lambda x: 1 if x == 'Yes' else 0)

    agg_dict = {
        'weight': ('weight', 'last'), # Take the last weight reported for the day
        'total_glasses_of_fluid': ('glasses_of_fluid', 'sum'), # Sum all fluid intake for the day
        'alcohol_consumed_daily_flag': ('alcohol_consumed_flag', 'max'), # 1 if alcohol was consumed at least once, 0 otherwise
    }

    # Add meal flags to aggregation dict using 'max'
    for meal in meal_types:
        agg_dict[f'had_{meal.lower()}'] = (f'had_{meal.lower()}', 'max')

    daily_reporting_metrics = reporting_df.groupby('dateTime').agg(
        **agg_dict
    ).reset_index()

    # 5. Handle NaNs in 'weight' after daily aggregation
    daily_reporting_metrics['weight'] = daily_reporting_metrics['weight'].interpolate(method='linear', limit_direction='both', limit_area='inside')
    daily_reporting_metrics['weight'] = daily_reporting_metrics['weight'].fillna(method='ffill').fillna(method='bfill') # Fill remaining NaNs at edges

    # Ensure boolean/int columns are correct type after aggregation
    for meal in meal_types:
        daily_reporting_metrics[f'had_{meal.lower()}'] = daily_reporting_metrics[f'had_{meal.lower()}'].astype(int)
    daily_reporting_metrics['alcohol_consumed_daily_flag'] = daily_reporting_metrics['alcohol_consumed_daily_flag'].astype(int)

    return daily_reporting_metrics

In [5]:
def preprocess_injury_data(participant_id):
    injury_df = pd.read_csv(f"./PMDATA/{participant_id}/pmsys/injury.csv")

    # Convert 'effective_time_frame' to datetime and extract date
    injury_df['effective_time_frame'] = pd.to_datetime(injury_df['effective_time_frame'])
    injury_df['injury_date'] = injury_df['effective_time_frame'].dt.date

    # Process 'injuries' column (which contains dictionaries)
    # Convert string representation of dict to actual dict for empty check
    injury_df['injuries_dict'] = injury_df['injuries'].apply(lambda x: json.loads(x.replace("'", '"')) if isinstance(x, str) else x)

    # Flag if any injury was reported for the entry
    injury_df['has_injury_reported_session'] = injury_df['injuries_dict'].apply(lambda x: 1 if x else 0)

    # Count the number of distinct injury areas reported in that session
    injury_df['num_injury_areas_session'] = injury_df['injuries_dict'].apply(lambda x: len(x) if x else 0)

    # Optional: Extract severity and specific locations if needed for detailed analysis
    # For simplicity, we'll stick to a count and a general flag for the dashboard.
    # If a specific injury (e.g., 'right_foot') is important, you can add flags:
    # injury_df['has_right_foot_injury'] = injury_df['injuries_dict'].apply(lambda x: 1 if 'right_foot' in x else 0)

    # Aggregate to daily level
    # Since it's weekly, and we just want to know if there was an injury report on a day
    daily_injury_metrics = injury_df.groupby('injury_date').agg(
        has_injury_reported=('has_injury_reported_session', 'max'), # Use max to get 1 if any injury was reported that day
        num_unique_injury_areas_daily=('num_injury_areas_session', 'max') # Max number of areas reported on that day
    ).reset_index()

    # Rename the date column for merging consistency
    daily_injury_metrics.rename(columns={'injury_date': 'dateTime'}, inplace=True)

    # Ensure has_injury_reported is int (0 or 1)
    daily_injury_metrics['has_injury_reported'] = daily_injury_metrics['has_injury_reported'].astype(int)

    return daily_injury_metrics

In [6]:
def get_daily_aggregate(participant_id,json_file_name, aggregate_type):
    specific = pd.read_json(f"./PMDATA/{participant_id}/fitbit/{json_file_name}.json")
    if(aggregate_type == "sum"):
        specific.rename(columns = {"value":f"{json_file_name}_sum"}, inplace = True)
        specific = specific.set_index("dateTime").resample("D").sum().reset_index()
    if(aggregate_type == "mean"):
        specific.rename(columns = {"value":f"{json_file_name}_mean"}, inplace = True)
        specific = specific.set_index("dateTime").resample("D").mean().reset_index()
    return specific
    
    
    

In [7]:
def deal_with_sleep_data_json(participant_id):
    sleep_df = pd.read_json(f"./PMDATA/{participant_id}/fitbit/sleep.json")
    sleep_df['dateOfSleep'] = pd.to_datetime(sleep_df['dateOfSleep'])
    sleep_df['startTime'] = pd.to_datetime(sleep_df['startTime'])
    sleep_df['endTime'] = pd.to_datetime(sleep_df['endTime'].str.replace('Z', '')) # Handle 'Z' if present
    def extract_sleep_levels(levels_data):
        if isinstance(levels_data, dict) and 'summary' in levels_data:
            summary = levels_data['summary']
            return {
                'deep_sleep_minutes': summary.get('deep', {}).get('minutes'),
                'light_sleep_minutes': summary.get('light', {}).get('minutes'),
                'rem_sleep_minutes': summary.get('rem', {}).get('minutes'),
                'awake_minutes_in_sleep': summary.get('wake', {}).get('minutes')
            }
        return {
            'deep_sleep_minutes': None,
            'light_sleep_minutes': None,
            'rem_sleep_minutes': None,
            'awake_minutes_in_sleep': None
        }
    sleep_levels_extracted = sleep_df['levels'].apply(extract_sleep_levels).apply(pd.Series)
    sleep_df = pd.concat([sleep_df, sleep_levels_extracted], axis=1)
    # Select and rename columns for clarity
    sleep_df_processed = sleep_df[[
        'logId', 'dateOfSleep', 'duration', 'minutesToFallAsleep', 'minutesAsleep',
        'minutesAwake', 'minutesAfterWakeup', 'timeInBed', 'efficiency',
        'deep_sleep_minutes', 'light_sleep_minutes', 'rem_sleep_minutes', 'awake_minutes_in_sleep'
    ]].copy()
    sleep_df_processed.rename(columns={'logId': 'sleep_log_entry_id'}, inplace=True)

    sleep_score_df = pd.read_csv(f"./PMDATA/{participant_id}/fitbit/sleep_score.csv")
    sleep_score_df['timestamp'] = pd.to_datetime(sleep_score_df['timestamp'].str.replace('Z', ''))
    sleep_score_df_processed = sleep_score_df[[
        'sleep_log_entry_id', 'timestamp', 'overall_score', 'composition_score',
        'revitalization_score', 'duration_score', 'restlessness'
    ]].copy()

    sleep_df_processed.drop_duplicates(inplace = True)
    sleep_score_df_processed.drop_duplicates(inplace = True)
    
    combined_sleep_df = pd.merge(
        sleep_df_processed,
        sleep_score_df_processed,
        on='sleep_log_entry_id',
        how='outer'
    )

    combined_sleep_df.rename(columns = {"dateOfSleep":"dateTime"}, inplace = True)

    return combined_sleep_df


def aggregate_daily_sleep_data(combined_sleep_df):
    """
    Aggregates the combined sleep DataFrame to a daily level.

    Args:
        combined_sleep_df (pd.DataFrame): DataFrame containing merged sleep.json and sleep_score.csv data.

    Returns:
        pd.DataFrame: Daily aggregated sleep metrics.
    """
    # Ensure 'dateTime' is of datetime.date type for grouping
    combined_sleep_df['dateTime'] = pd.to_datetime(combined_sleep_df['dateTime']).dt.date

    # Define aggregation dictionary for daily summary
    daily_sleep_metrics = combined_sleep_df.groupby('dateTime').agg(
        # Sums for durations/minutes (assuming multiple sleep sessions like naps per day)
        total_sleep_duration_minutes=('duration', 'sum'),
        total_minutes_asleep=('minutesAsleep', 'sum'),
        total_minutes_awake_during_sleep=('minutesAwake', 'sum'),
        total_time_in_bed_minutes=('timeInBed', 'sum'),
        total_deep_sleep_minutes=('deep_sleep_minutes', 'sum'),
        total_light_sleep_minutes=('light_sleep_minutes', 'sum'),
        total_rem_sleep_minutes=('rem_sleep_minutes', 'sum'),
        total_awake_minutes_in_sleep_stages=('awake_minutes_in_sleep', 'sum'),

        # Averages for scores and single-session metrics
        avg_minutes_to_fall_asleep=('minutesToFallAsleep', 'mean'), # Avg across sessions for the day
        avg_minutes_after_wakeup=('minutesAfterWakeup', 'mean'),   # Avg across sessions for the day
        avg_sleep_efficiency=('efficiency', 'mean'),
        avg_overall_sleep_score=('overall_score', 'mean'),
        avg_composition_score=('composition_score', 'mean'),
        avg_revitalization_score=('revitalization_score', 'mean'),
        avg_duration_score=('duration_score', 'mean'),
        avg_restlessness=('restlessness', 'mean')
    ).reset_index()

    # Fill NaNs for sum-based metrics with 0 (if no sleep data for a given day)
    # The default behavior of sum() is to ignore NaNs, so if there's no data for a day,
    # the sum will naturally be 0 after grouping.
    # We can ensure this for clarity.
    daily_sleep_metrics.fillna({
        'total_sleep_duration_minutes': 0,
        'total_minutes_asleep': 0,
        'total_minutes_awake_during_sleep': 0,
        'total_time_in_bed_minutes': 0,
        'total_deep_sleep_minutes': 0,
        'total_light_sleep_minutes': 0,
        'total_rem_sleep_minutes': 0,
        'total_awake_minutes_in_sleep_stages': 0,
    }, inplace=True)

    # Rename the date column to a common name for final merging with other dataframes
    daily_sleep_metrics.rename(columns={'dateTime': 'dateTime'}, inplace=True)

    return daily_sleep_metrics

In [8]:
def preprocess_hr_zones(participant_id):
    hr_zones_df = pd.read_json(f"./PMDATA/{participant_id}/fitbit/time_in_heart_rate_zones.json")
    hr_zones_df['dateTime'] = pd.to_datetime(hr_zones_df['dateTime'])

    def extract_hr_zones(value_data):
        if isinstance(value_data, dict) and 'valuesInZones' in value_data:
            zones = value_data['valuesInZones']
            return {
                'time_in_below_default_zone1_minutes': zones.get('BELOW_DEFAULT_ZONE_1', 0.0),
                'time_in_fat_burn_zone_minutes': zones.get('IN_DEFAULT_ZONE_1', 0.0), # Fat Burn Zone 
                'time_in_cardio_zone_minutes': zones.get('IN_DEFAULT_ZONE_2', 0.0),   # Cardio Zone 
                'time_in_peak_zone_minutes': zones.get('IN_DEFAULT_ZONE_3', 0.0)     # Peak Zone 
            }
        # Return None or appropriate default if 'valuesInZones' is missing or not a dict
        return {
            'time_in_below_default_zone1_minutes': None,
            'time_in_fat_burn_zone_minutes': None,
            'time_in_cardio_zone_minutes': None,
            'time_in_peak_zone_minutes': None
        }

    hr_zones_extracted = hr_zones_df['value'].apply(extract_hr_zones).apply(pd.Series)

    hr_zones_df_processed = pd.concat([hr_zones_df['dateTime'], hr_zones_extracted], axis=1)
        
    hr_zones_df_processed['dateTime'] = hr_zones_df_processed['dateTime'].dt.date

    return hr_zones_df_processed

    


In [9]:
def preprocess_heart_rate(participant_id,json_file_name,key_name = "bpm"): 
    heart_rate_df = pd.read_json(f"./PMDATA/{participant_id}/fitbit/{json_file_name}.json")
    heart_rate_df["heart_rate_value"] = heart_rate_df["value"].apply(lambda x: x[key_name])
    heart_rate_df = heart_rate_df.drop("value", axis = 1)
    daily_heart_rate = heart_rate_df.set_index("dateTime").resample("D").mean().reset_index()
    daily_heart_rate.rename(columns = {"heart_rate_value":"avg_daily_heart_rate"}, inplace = True)
    daily_heart_rate["dateTime"] = daily_heart_rate["dateTime"].dt.date
    return daily_heart_rate

In [10]:
def preprocess_exercise_data(participant_id):
    exercise_df = pd.read_json(f"./PMDATA/{participant_id}/fitbit/exercise.json")

    # Convert 'startTime' to datetime and extract date
    exercise_df['startTime'] = pd.to_datetime(exercise_df['startTime'])
    exercise_df['exercise_date'] = exercise_df['startTime'].dt.date

    selected_exercise_df = exercise_df[[
        "exercise_date",
        "activityName",
        "calories",  # Added 'calories' as per your confirmation
        "duration",
        "steps",
        "distance",
        "averageHeartRate"
    ]].copy()

    daily_exercise_metrics = selected_exercise_df.groupby('exercise_date').agg(
            total_exercise_calories=('calories', 'sum'),
            total_exercise_duration_minutes=('duration', 'sum'),
            total_exercise_steps=('steps', 'sum'),
            total_exercise_distance_km=('distance', 'sum'),
            avg_exercise_heart_rate=('averageHeartRate', 'mean'), # Mean of average HR across sessions
            num_exercise_sessions=('activityName', 'count') # Count of sessions per day
        ).reset_index()
    daily_exercise_metrics.fillna({
        'total_exercise_calories': 0,
        'total_exercise_duration_minutes': 0,
        'total_exercise_steps': 0,
        'total_exercise_distance_km': 0,
    }, inplace=True)

    daily_exercise_metrics.rename(columns = {"exercise_date": "dateTime"}, inplace = True)

    return daily_exercise_metrics

## Heart Rate Data

In [11]:
participant_id= "p07"

In [12]:
path = "./PMDATA/p07/fitbit/"

In [13]:
p07_daily_heart_rate_df = preprocess_heart_rate(participant_id,"heart_rate")

In [14]:
resting_heart_rate_df = preprocess_heart_rate(participant_id, "resting_heart_rate",key_name="value")

In [15]:
hr_zones_df = preprocess_hr_zones(participant_id)

In [16]:
distance_df = get_daily_aggregate(participant_id,"distance","sum")
lightly_active_minutes_df = get_daily_aggregate(participant_id,"lightly_active_minutes","sum")
moderately_active_minutes_df = get_daily_aggregate(participant_id,"moderately_active_minutes","sum")
sedentary_minutes_df = get_daily_aggregate(participant_id,"sedentary_minutes","sum")
very_active_minutes_df = get_daily_aggregate(participant_id,"very_active_minutes","sum")
steps_df = get_daily_aggregate(participant_id,"steps","sum")

In [17]:
sleep_df = deal_with_sleep_data_json(participant_id)
sleep_df  = aggregate_daily_sleep_data(sleep_df)

In [18]:
exercise_df = preprocess_exercise_data(participant_id)

In [19]:
wellness_df = preprocess_wellness_csv(participant_id)

In [20]:
srpe_df = preprocess_srpe_data(participant_id)

In [21]:
injury_df = preprocess_injury_data(participant_id)

In [22]:
reporting_df = preprocess_reporting_data(participant_id)

/tmp/ipykernel_1396/1381861460.py:32: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  daily_reporting_metrics['weight'] = daily_reporting_metrics['weight'].fillna(method='ffill').fillna(method='bfill') # Fill remaining NaNs at edges


In [23]:
all_dfs = [wellness_df,reporting_df, injury_df, srpe_df,exercise_df, 
           sleep_df, hr_zones_df, steps_df, very_active_minutes_df, sedentary_minutes_df,
          moderately_active_minutes_df,lightly_active_minutes_df,distance_df, hr_zones_df,
          resting_heart_rate_df, resting_heart_rate_df, p07_daily_heart_rate_df]

In [60]:
def merge_dfs(all_dfs):
    master_df = all_dfs[0].copy()
    for i in range(1,len(all_dfs)):
        master_df = pd.merge(master_df, all_dfs[i], on = "dateTime",how = "outer")

    master_df.drop_duplicates(subset=['dateTime'],inplace = True)
    return master_df

def convert_column_to_datetime(all_dfs):
    for i in range(len(all_dfs)):
        all_dfs[i]["dateTime"] = pd.to_datetime(all_dfs[i]["dateTime"])

    return all_dfs

In [61]:
all_dfs = convert_column_to_datetime(all_dfs)
master_df = merge_dfs(all_dfs)
master_df.set_index('dateTime', inplace=True)
master_df.sort_index(inplace=True)

In [63]:
master_df.isna().sum()

fatigue                                   16
mood                                      16
readiness                                 16
sleep_duration_h                          16
sleep_quality                             16
soreness                                  16
stress                                    16
weight                                    30
total_glasses_of_fluid                    30
alcohol_consumed_daily_flag               30
had_breakfast                             30
had_lunch                                 30
had_dinner                                30
had_evening                               30
has_injury_reported                      134
num_unique_injury_areas_daily            134
total_daily_training_load_srpe            45
total_daily_duration_min                  45
avg_daily_perceived_exertion              45
num_daily_srpe_sessions                   45
unique_daily_activity_names               45
total_exercise_calories                   24
total_exer

In [64]:
master_df["avg_daily_heart_rate"]

dateTime
2019-11-05           NaN
2019-11-06     92.286210
2019-11-07     80.636287
2019-11-08     63.418695
2019-11-09    109.914619
                 ...    
2020-03-27     74.001145
2020-03-28     70.408729
2020-03-29     64.688234
2020-03-30     67.115377
2020-03-31     80.959281
Name: avg_daily_heart_rate, Length: 148, dtype: float64

In [65]:
drop_columns = [
    "time_in_below_default_zone1_minutes_y",
    "time_in_fat_burn_zone_minutes_y",
    "time_in_cardio_zone_minutes_y",
    "time_in_peak_zone_minutes_y",
    "avg_daily_heart_rate_y",
    "avg_daily_heart_rate_x"
]

In [66]:
master_df.drop(drop_columns, axis = 1, inplace = True)

In [67]:
master_df.isna().sum()

fatigue                                   16
mood                                      16
readiness                                 16
sleep_duration_h                          16
sleep_quality                             16
soreness                                  16
stress                                    16
weight                                    30
total_glasses_of_fluid                    30
alcohol_consumed_daily_flag               30
had_breakfast                             30
had_lunch                                 30
had_dinner                                30
had_evening                               30
has_injury_reported                      134
num_unique_injury_areas_daily            134
total_daily_training_load_srpe            45
total_daily_duration_min                  45
avg_daily_perceived_exertion              45
num_daily_srpe_sessions                   45
unique_daily_activity_names               45
total_exercise_calories                   24
total_exer

In [68]:
master_df.head()

,fatigue,mood,readiness,sleep_duration_h,sleep_quality,soreness,stress,weight,total_glasses_of_fluid,alcohol_consumed_daily_flag,...,time_in_fat_burn_zone_minutes_x,time_in_cardio_zone_minutes_x,time_in_peak_zone_minutes_x,steps_sum,very_active_minutes_sum,sedentary_minutes_sum,moderately_active_minutes_sum,lightly_active_minutes_sum,distance_sum,avg_daily_heart_rate
dateTime,,,,,,,,,,,,,,,,,,,,,
2019-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,1440,0,0,NaN,NaN
2019-11-06,3.0,3.0,2.0,9.0,4.0,3.0,5.0,NaN,NaN,NaN,...,85.0,13.0,0.0,9370.0,65,1270,23,82,681100.0,92.286210
2019-11-07,4.0,3.0,6.0,9.0,4.0,3.0,4.0,NaN,NaN,NaN,...,115.0,29.0,0.0,19979.0,105,563,21,226,1455450.0,80.636287
2019-11-08,3.0,3.0,4.0,8.0,4.0,3.0,3.0,NaN,NaN,NaN,...,55.0,0.0,0.0,9614.0,29,660,9,224,705490.0,63.418695
2019-11-09,4.0,4.0,8.0,6.0,4.0,3.0,4.0,68.0,10.0,0.0,...,173.0,116.0,55.0,28445.0,324,383,16,243,2010980.0,109.914619


In [69]:
master_df.to_csv("./cleaned_data/p07.csv")

In [62]:
master_df[master_df.index.duplicated()]

,fatigue,mood,readiness,sleep_duration_h,sleep_quality,soreness,stress,weight,total_glasses_of_fluid,alcohol_consumed_daily_flag,...,moderately_active_minutes_sum,lightly_active_minutes_sum,distance_sum,time_in_below_default_zone1_minutes_y,time_in_fat_burn_zone_minutes_y,time_in_cardio_zone_minutes_y,time_in_peak_zone_minutes_y,avg_daily_heart_rate_x,avg_daily_heart_rate_y,avg_daily_heart_rate
dateTime,,,,,,,,,,,,,,,,,,,,,
